<a href="https://colab.research.google.com/github/Xalp/NLP1d/blob/master/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Xalp/NLP1d

Cloning into 'NLP1d'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 1), reused 17 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [ ]:
import os
os.chdir("/content/NLP1d")

## Part 1

In [ ]:
# Emission probability
from collections import defaultdict
import numpy as np
def emission(train):
    '''
    x: str, word
    y: str, tag
    train: training set path
    '''
    count_y_x = defaultdict(float)
    count_y  = defaultdict(float)
    emission_probs = defaultdict(float)
    dict_f = defaultdict()
    with open(train) as f:
        for line in f.readlines():
            if line == '\n': continue
            x, y = line.split()
            count_y_x[(y,x)] += 1
    
    for (y,x), cnt in count_y_x.items():
        count_y[y] += cnt
    for (y,x) in count_y_x.keys():
        e_x_y = count_y_x[(y,x)]/count_y[y]
        emission_probs[(x,y)] = e_x_y
        str_x_y = 'emission:'+str(y)+'+'+str(x)
        dict_f[str_x_y] = np.log(e_x_y)

    return emission_probs, dict_f


In [ ]:
import numpy as np
# Transition probability
def transition(train):
  with open(train,"r") as f:
    data = f.read().rstrip().splitlines()

  count_bigram = {}
  count_unigram = {}

  count_unigram["START"] = 0
  count_unigram["STOP"] = 0

  for i in range(len(data)):
    inst = data[i]
    # if this is an empty line:
    if len(inst) == 0:
      # building with previous inst
      tag = "STOP"
      if i != 0:
        prev_inst = data[i-1]
        prev_tag = prev_inst.split()[-1]
        if (tag, prev_tag) not in count_bigram.keys():
          count_bigram[(tag, prev_tag)] = 1
        else:
          count_bigram[(tag, prev_tag)] += 1
      # building with the next inst
      tag = "START"
      if i != len(data) - 1:
        next_inst = data[i+1]
        next_tag = next_inst.split()[-1]
        if (tag, next_tag) not in count_bigram.keys():
          count_bigram[(tag, next_tag)] = 1
        else:
          count_bigram[(tag, next_tag)] += 1
      # increase the count of start and stop
      count_unigram["START"] += 1
      count_unigram["STOP"] += 1
      continue
    # do in all cases:
    tag = inst.split()[-1]
    if tag not in count_unigram.keys():
      count_unigram[tag] = 1
    else:
      count_unigram[tag] += 1
    # if not the last inst:
    if i != len(data) - 1:
      next_inst = data[i+1]
      if len(next_inst) == 0:
        continue
      next_tag = next_inst.split()[-1]
      if (tag, next_tag) not in count_bigram.keys():
        count_bigram[(tag, next_tag)] = 1
      else:
        count_bigram[(tag, next_tag)] += 1

  # calculate transition matrix
  f = {}
  q = {}
  for (y_1, y_2), count_bi in count_bigram.items():
    _str_ = "transition:" + y_1 + "+" + y_2
    q[(y_1, y_2)] = count_bi / count_unigram[y_1]
    if count_bi == 0:
      f[_str_] = - np.inf
    else:
      f[_str_] = np.log(count_bi / count_unigram[y_1])
  return q, f



In [ ]:
e,f = emission("/content/NLP1d/dataset/train")
q,f1 = transition("/content/NLP1d/dataset/train")
f.update(f1)

## Part 2

In [ ]:
def score_sequence(x,y,dict_transition,dict_emission):
  y.append("STOP")
  y.insert(0,"START")
  sum=0
  for i in range(1,len(y)):
    _str_ = "transition:" + y[i-1] + "+" + y[i]
    sum += dict_transition[_str_]
    if i != (len(y)-1):
      _str_ = "emission:" + y[i] + "+" + x[i-1]
      score += dict_emission[_str_]
  return sum